In [26]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import socket
import glob
import os
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
from torch.utils.data import TensorDataset, DataLoader
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
if socket.gethostname() == 'Rohits-MacBook-Pro.local':
    rootdir = '/Users/rohitchanne/Documents/capstone/data/data_parquet/'
else: 
    rootdir = '' # Enter your hone dir here

In [3]:
list_of_files = filter( os.path.isfile, glob.glob(rootdir + '*') )
files_with_size = [ file_path for file_path in list_of_files ]

In [4]:
dfs_parquet = {}
for file_path in files_with_size:
    if 'parquet' in file_path:
        file_name = file_path.split('/')[-1]
        df_name = file_name.split('_')[0]
        print(f'Reading Data File: {file_name}')       
        dfs_parquet[df_name] = pd.read_parquet(file_path, engine='pyarrow')

Reading Data File: Wednesday-21-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-23-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thuesday-20-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-22-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-16-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-28-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Wednesday-14-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-15-02-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Thursday-01-03-2018_TrafficForML_CICFlowMeter_clean.parquet
Reading Data File: Friday-02-03-2018_TrafficForML_CICFlowMeter_clean.parquet


In [5]:
for k,df in dfs_parquet.items():
    print(k)
    df['is_allowed'] = df['Label'] == 'Benign'
    del df['Label']
    print(f'DF:{k}, Shape{df.shape}')

Wednesday-21-02-2018
DF:Wednesday-21-02-2018, Shape(1048575, 80)
Friday-23-02-2018
DF:Friday-23-02-2018, Shape(1048575, 80)
Thuesday-20-02-2018
DF:Thuesday-20-02-2018, Shape(7948748, 84)
Thursday-22-02-2018
DF:Thursday-22-02-2018, Shape(1048575, 80)
Friday-16-02-2018
DF:Friday-16-02-2018, Shape(1048574, 80)
Wednesday-28-02-2018
DF:Wednesday-28-02-2018, Shape(613071, 80)
Wednesday-14-02-2018
DF:Wednesday-14-02-2018, Shape(1048575, 80)
Thursday-15-02-2018
DF:Thursday-15-02-2018, Shape(1048575, 80)
Thursday-01-03-2018
DF:Thursday-01-03-2018, Shape(331100, 80)
Friday-02-03-2018
DF:Friday-02-03-2018, Shape(1048575, 80)


In [6]:
from sklearn.preprocessing import MinMaxScaler
cat_cols = ['Dst Port']

In [7]:
df_train_rnn = pd.concat([dfs_parquet['Friday-02-03-2018'], 
                      dfs_parquet['Thursday-22-02-2018']
                     ]
                    )
#Transforming timestamp and category columns and also scaling data using minmax scalar
df_train_rnn['Timestamp'] = pd.to_datetime(df_train_rnn['Timestamp'])
df_train_rnn['Date'] = pd.to_datetime(df_train_rnn['Timestamp']).dt.date
df_train_rnn['TS_relative'] = (df_train_rnn['Timestamp'].astype(int) - 
                             pd.to_datetime(df_train_rnn['Date']).astype(int))/ 10**9
df_train_rnn = df_train_rnn.drop(['Timestamp'], axis = 1)
df_train_rnn = df_train_rnn.drop(['Date'], axis = 1)
df_train_rnn.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train_rnn.fillna(0, inplace=True)
df_train_rnn[cat_cols] = df_train_rnn[cat_cols].astype('category')
X_train = df_train_rnn.drop(['is_allowed'], axis = 1)
y_train = df_train_rnn['is_allowed']*1 
stdsc = MinMaxScaler()
stdsc.fit(X_train)
X_train_scaled = stdsc.transform(X_train)
x_for_nn =  torch.from_numpy(X_train_scaled).float()
x_for_nn.shape
y_for_nn = torch.from_numpy(y_train.to_numpy())
y_for_nn.shape

torch.Size([2097150])

In [8]:
num_classes = 2
num_epochs = 2
batch_size = 32768
learning_rate = 0.001

input_size = 79
sequence_length = 28
hidden_size = 128
num_layers = 2
device = torch.device("cpu")

# RNN model: GRU Implementation

In [9]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

model = GRUNet(input_dim= input_size, hidden_dim = hidden_size, output_dim = 1, n_layers = num_layers).to(device)

In [54]:
x_for_nn.shape

torch.Size([2097150, 79])

In [27]:
batch_size = 32768
train_data = TensorDataset(x_for_nn, y_for_nn)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
model.train()
for epoch in range(num_epochs):
        start_time = time.process_time()
        h = model.init_hidden(batch_size)
        print(h.shape)
        avg_loss = 0.
        counter = 0
        for x, y in train_loader:
            counter += 1
            h = h.data
            model.zero_grad() 
            print(x.shape)
            x = x.view([batch_size, -1, input_size]).to(device)
            y = y.to(device)
            y = y.unsqueeze(1)
            out, h = model(x.float(), h)
            loss = criterion(out, y.float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            print(f"Epoch {epoch}, Batch {counter}...... Average Loss for Epoch: {avg_loss/counter}")
            _, predicted = torch.max(out.data, 1)
            print(f'F1: {f1_score(y, predicted)}')
            print(f'Accuracy: {accuracy_score(y, predicted)}')
            print(f'Precision: {precision_score(y, predicted)}')
            print(f'Recall: {recall_score(y, predicted)}') 
        current_time = time.process_time()

torch.Size([2, 32768, 128])
torch.Size([32768, 79])
Epoch 0, Batch1...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13714599609375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 0, Batch2...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.135406494140625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 0, Batch3...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1376953125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 0, Batch4...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.140411376953125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 0, Batch5...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.14105224609375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 0, Batch6...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.134979248046875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 0, Batch7...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13824462890625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79

torch.Size([32768, 79])
Epoch 0, Batch61...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13433837890625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 0, Batch62...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13629150390625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 0, Batch63...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.139068603515625
Precision: 0.0
Recall: 0.0
torch.Size([2, 32768, 128])
torch.Size([32768, 79])
Epoch 1, Batch1...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13525390625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 1, Batch2...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.136932373046875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 1, Batch3...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13800048828125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 1, Batch4...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.136566162109375
Precision: 0.0
Recall: 0.0
torch.Size([32768

torch.Size([32768, 79])
Epoch 1, Batch58...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.134613037109375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 1, Batch59...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.139404296875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 1, Batch60...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13922119140625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 1, Batch61...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1376953125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 1, Batch62...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1385498046875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 1, Batch63...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.133392333984375
Precision: 0.0
Recall: 0.0
torch.Size([2, 32768, 128])
torch.Size([32768, 79])
Epoch 2, Batch1...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13519287109375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 

torch.Size([32768, 79])
Epoch 2, Batch55...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13433837890625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 2, Batch56...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.139251708984375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 2, Batch57...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1390380859375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 2, Batch58...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13525390625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 2, Batch59...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13525390625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 2, Batch60...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.138763427734375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 2, Batch61...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13818359375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 2, Batch62...... Av

torch.Size([32768, 79])
Epoch 3, Batch52...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.134918212890625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 3, Batch53...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.138336181640625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 3, Batch54...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13665771484375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 3, Batch55...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.137542724609375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 3, Batch56...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.136993408203125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 3, Batch57...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.136260986328125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 3, Batch58...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1365966796875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 3, Batc

...

torch.Size([32768, 79])
Epoch 5, Batch46...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13409423828125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 5, Batch47...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.136474609375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 5, Batch48...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.131561279296875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 5, Batch49...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13848876953125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 5, Batch50...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13714599609375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 5, Batch51...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1363525390625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 5, Batch52...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13726806640625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 5, Batch53...

torch.Size([32768, 79])
Epoch 6, Batch43...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.135650634765625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 6, Batch44...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.134490966796875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 6, Batch45...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13739013671875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 6, Batch46...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13861083984375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 6, Batch47...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13616943359375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 6, Batch48...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.137451171875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 6, Batch49...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1363525390625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 6, Batch50..

torch.Size([32768, 79])
Epoch 7, Batch40...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13555908203125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 7, Batch41...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.132354736328125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 7, Batch42...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1365966796875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 7, Batch43...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.135589599609375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 7, Batch44...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13397216796875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 7, Batch45...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.136932373046875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 7, Batch46...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13677978515625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 7, Batch4

torch.Size([32768, 79])
Epoch 8, Batch37...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13775634765625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 8, Batch38...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.137237548828125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 8, Batch39...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13665771484375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 8, Batch40...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1348876953125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 8, Batch41...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13824462890625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 8, Batch42...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.135986328125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 8, Batch43...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.136199951171875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 8, Batch44..

torch.Size([32768, 79])
Epoch 9, Batch34...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.14093017578125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 9, Batch35...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1395263671875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 9, Batch36...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.137847900390625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 9, Batch37...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.137603759765625
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 9, Batch38...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.139495849609375
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 9, Batch39...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1387939453125
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 9, Batch40...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.1357421875
Precision: 0.0
Recall: 0.0
torch.Size([32768, 79])
Epoch 9, Batch41....

In [17]:
x_for_nn.shape[0]

2097150

In [34]:
batch_size=x_for_nn.shape[0]
train_loader = DataLoader(train_data, shuffle=True, batch_size=x_for_nn.shape[0], drop_last=True)

### Running with complete data

In [35]:
num_epochs = 4
for epoch in range(num_epochs):
        start_time = time.process_time()
        h = model.init_hidden(batch_size)
        print(h.shape)
        avg_loss = 0.
        counter = 0
        for x, y in train_loader:
            counter += 1
            h = h.data
            model.zero_grad() 
            print(x.shape)
            x = x.view([batch_size, -1, input_size]).to(device)
            y = y.to(device)
            y = y.unsqueeze(1)
            out, h = model(x.float(), h)
            loss = criterion(out, y.float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            print(f"Epoch {epoch}...... Average Loss for Epoch: {avg_loss/counter}")
            _, predicted = torch.max(out.data, 1)
            print(f'F1: {f1_score(y, predicted)}')
            print(f'Accuracy: {accuracy_score(y, predicted)}')
            print(f'Precision: {precision_score(y, predicted)}')
            print(f'Recall: {recall_score(y, predicted)}')            
        current_time = time.process_time()

torch.Size([2, 2097150, 128])
torch.Size([2097150, 79])
Epoch 0...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13663924850392198
Precision: 0.0
Recall: 0.0
torch.Size([2, 2097150, 128])
torch.Size([2097150, 79])
Epoch 1...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13663924850392198
Precision: 0.0
Recall: 0.0
torch.Size([2, 2097150, 128])
torch.Size([2097150, 79])
Epoch 2...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13663924850392198
Precision: 0.0
Recall: 0.0
torch.Size([2, 2097150, 128])
torch.Size([2097150, 79])
Epoch 3...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13663924850392198
Precision: 0.0
Recall: 0.0


# RNN model: LSTM Implementation

In [32]:
class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [36]:
model = LSTMNet(input_dim= input_size, hidden_dim = hidden_size, output_dim = 1, n_layers = num_layers).to(device)

In [39]:
num_epochs = 4
for epoch in range(num_epochs):
        start_time = time.process_time()
        h = model.init_hidden(batch_size)
        #print(h.shape)
        avg_loss = 0.
        counter = 0
        for x, y in train_loader:
            counter += 1
            h = tuple([e.data for e in h])
            model.zero_grad() 
            print(x.shape)
            x = x.view([batch_size, -1, input_size]).to(device)
            y = y.to(device)
            y = y.unsqueeze(1)
            out, h = model(x.float(), h)
            loss = criterion(out, y.float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            print(f"Epoch {epoch}...... Average Loss for Epoch: {avg_loss/counter}")
            _, predicted = torch.max(out.data, 1)
            print(f'F1: {f1_score(y, predicted)}')
            print(f'Accuracy: {accuracy_score(y, predicted)}')
            print(f'Precision: {precision_score(y, predicted)}')
            print(f'Recall: {recall_score(y, predicted)}')            
        current_time = time.process_time()

torch.Size([2097150, 79])
Epoch 0...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13663924850392198
Precision: 0.0
Recall: 0.0
torch.Size([2097150, 79])
Epoch 1...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13663924850392198
Precision: 0.0
Recall: 0.0
torch.Size([2097150, 79])
Epoch 2...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13663924850392198
Precision: 0.0
Recall: 0.0
torch.Size([2097150, 79])
Epoch 3...... Average Loss for Epoch: 0.0
F1: 0.0
Accuracy: 0.13663924850392198
Precision: 0.0
Recall: 0.0
